In [4]:
from cobra.io import read_sbml_model, write_sbml_model
from mewpy import add_enzyme_constraints
from mewpy.simulation import get_simulator
from mewpy.simulation.environment import Environment as Environment
from mewpy.util.request import retreive_gene
import json

ImportError: cannot import name 'add_enzyme_constraints' from 'mewpy' (/home/mmartins/anaconda3/envs/mcd22/lib/python3.9/site-packages/mewpy/__init__.py)

In [2]:
model = read_sbml_model("models/iAF1260.xml")
sim = get_simulator(model)
sim.set_objective("BIOMASS")

No objective coefficients in model. Unclear what should be optimized


NameError: name 'get_simulator' is not defined

In [3]:
sim.simulate()

objective: 28.215758805752888
Status: OPTIMAL
Constraints: OrderedDict()
Method:SimulationMethod.FBA

### Retreive MW infos

The protein IDs and MW (Da) are obtained from UniProt.

Note: does not retreive Kcats from BRENDA. All catalytic rates are equal to 100000/s 

In [4]:
data_all = dict()
try:
    with open(sim.id+".json", 'r') as f:
        data = json.load(f)
except:
    data = dict()
    for gene in sim.genes:
        try:
            info = retreive_gene(gene,"Escherichia coli")
            # set the inverse kcat to a low value
            data[gene]={'protein':info['protein'],'mw':info['mw']*0.001,'kcat':1e-5}
            data_all[gene]=info
        except:
            data[gene]={'protein': gene[len(sim._g_prefix):], 'mw': 0, 'kcat': 0}
    with open(sim.id+".json", "w") as f:
        json.dump(data,f)

No comments
No comments


In [5]:
print("Genes with missing data:")
for k,v in data.items():
    if v['mw']==0:
        print(k)

Genes with missing data:
b1692


### Build the ec model

In [6]:
ec_sim = add_enzyme_constraints(sim, data=data)

Splitting isozymes:   0%|                                                                                                                                                         | 0/3239 [00:00<?, ?it/s]/home/vmsapereira/anaconda3/lib/python3.9/site-packages/cobra/core/group.py:148: UserWarning: need to pass in a list
  warn("need to pass in a list")
Adding proteins usage to reactions: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6548/6548 [04:02<00:00, 26.96it/s]


### Protein pool usage

In [7]:
res = ec_sim.simulate()
print(res)

objective: 28.215758805752927
Status: OPTIMAL
Constraints: OrderedDict()
Method:SimulationMethod.FBA


In [8]:
res.find('pool')

,Flux rate
Reaction ID,
prot_pool_exchange,12.729624


### Save the model

In [9]:
write_sbml_model(ec_sim.model,"eciAF1260.xml")

In [10]:
env = Environment.complete(ec_sim, max_uptake=1000.0, inplace=False)

In [11]:
env['prot_pool_exchange']=(0,0.8)

In [12]:
ec_sim.simulate(constraints=env)

objective: 4.4311771722358655
Status: OPTIMAL
Constraints: OrderedDict([('EX_g3ps_e', (-1000.0, inf)), ('EX_cynt_e', (-1000.0, inf)), ('EX_acac_e', (-1000.0, inf)), ('EX_indole_e', (-1000.0, inf)), ('EX_inost_e', (-1000.0, inf)), ('EX_3amp_e', (-1000.0, inf)), ('EX_xmp_e', (-1000.0, inf)), ('EX_lipa_cold_e', (-1000.0, inf)), ('EX_ppal_e', (-1000.0, inf)), ('EX_adocbl_e', (-1000.0, inf)), ('EX_hom__L_e', (-1000.0, inf)), ('EX_dcyt_e', (-1000.0, inf)), ('EX_kdo2lipid4_e', (-1000.0, inf)), ('EX_cys__L_e', (-1000.0, inf)), ('EX_ppt_e', (-1000.0, inf)), ('EX_gthox_e', (-1000.0, inf)), ('EX_anhgm_e', (-1000.0, inf)), ('EX_maltpt_e', (-1000.0, inf)), ('EX_pacald_e', (-1000.0, inf)), ('EX_ocdcea_e', (-1000.0, inf)), ('EX_galctn__L_e', (-1000.0, inf)), ('EX_ura_e', (-1000.0, inf)), ('EX_tcynt_e', (-1000.0, inf)), ('EX_lyx__L_e', (-1000.0, inf)), ('EX_no3_e', (-1000.0, inf)), ('EX_3hcinnm_e', (-1000.0, inf)), ('EX_skm_e', (-1000.0, inf)), ('EX_cpgn_un_e', (-1000.0, inf)), ('EX_tym_e', (-1000.0, 

In [13]:
data_all

{'b0002': {'organism': 'Escherichia coli (strain K12)',
  'protein': 'P00561',
  'gene': 'b0002',
  'locus': ['b0002', 'JW0001'],
  'synonyms': 'thrA1 thrA2',
  'properties': {'Catalytic Activity': [('L-homoserine + NADP(+) = H(+) + L-aspartate 4-semialdehyde + NADPH',
     '1.1.1.3'),
    ('L-homoserine + NAD(+) = H(+) + L-aspartate 4-semialdehyde + NADH',
     '1.1.1.3'),
    ('ATP + L-aspartate = 4-phospho-L-aspartate + ADP', '2.7.2.4')],
   'Regulation Activity': 'The enzyme activities are regulated allosterically by L-threonine',
   'Pathway': 'Amino-acid biosynthesis; L-threonine biosynthesis; L-threonine from L-aspartate: step 3/5'},
  'seq': 'MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNHLVAMIEKTISGQDALPNISDAERIFAELLTGLAAAQPGFPLAQLKTFVDQEFAQIKHVLHGISLLGQCPDSINAALICRGEKMSIAIMAGVLEARGHNVTVIDPVEKLLAVGHYLESTVDIAESTRRIAASRIPADHMVLMAGFTAGNEKGELVVLGRNGSDYSAAVLAACLRADCCEIWTDVDGVYTCDPRQVPDARLLKSMSYQEAMELSYFGAKVLHPRTITPIAQFQIPCLIKNTGNPQAPGTLIGASRDEDELPVKGISNLNNMAMFSVSGPGMKGMVGMAARVFAAMSR